In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from transformers import AutoModel, AutoTokenizer
import pandas as pd

import tqdm

In [2]:
# load datasets
df = pd.read_csv('../data/hackathon_train.csv', encoding='cp949', index_col=0)
df.index = range(len(df))
df.tail()

,User_ID,Gender,Age,MBTI,Q_number,Answer
11515,240,0,40,ISTJ,44,<그렇다> 저는 계획에 차질이 생기면 돌아가기 위해 노력을 합니다. 이유는 그 계획...
11516,240,0,40,ISTJ,45,<그렇다> 저는 예전의 실수를 후회할 때가 많습니다. 이유는 그만큼 나태하게 산 적...
11517,240,0,40,ISTJ,46,<아니다> 저는 인간의 존재와 삶의 이유에 대해 깊이 생각하지 않습니다. 이유는 이...
11518,240,0,40,ISTJ,47,<아니다> 저는 감정에 휘둘리는 편이 아닙니다. 이유는 감정을 감추고 밖으로 표현하...
11519,240,0,40,ISTJ,48,<아니다> 저는 상대방 잘못일 때 상대방의 체면을 살려주기 위해 노력하지 않습니다....


In [3]:
dq = pd.read_excel('../data/Question.xlsx', index_col=0)
dq.tail()

,index.1,Question,Positive,Negative
index,,,,
56,56,단계를 건너뛰는 일 없이 절차대로 일을 완수하는 편인가요? 그러한 최근 경험은 어떤...,단계를 건너뛰는 일 없이 절차대로 일을 완수하는 편이에요.,단계를 건너뛰는 일 없이 절차대로 일을 완수하는 편은 아니에요.
57,57,논란이 되거나 논쟁을 불러일으킬 수 있는 주제에 관심이 많나요? 최근의 사례를 말씀...,논란이 되거나 논쟁을 불러일으킬 수 있는 주제에 관심이 많아요.,논란이 되거나 논쟁을 불러일으킬 수 있는 주제에는 관심이 많지 않아요.
58,58,자신보다 다른 사람에게 더 필요한 기회라고 생각되면 기회를 포기할 수도 있나요? 이...,저보다 다른 사람에게 더 필요한 기회라고 생각되면 기회를 포기할 수도 있어요.,저보다 다른 사람에게 더 필요한 기회라 해도 기회를 포기할 수는 없어요.
59,59,마감 기한을 지키기가 힘든가요? 경험을 이야기해보아요.,마감 기한을 지키기가 힘들어요.,마감 기한을 지키기가 힘들지 않아요.
60,60,일이 원하는 대로 진행될 것이라는 자신감이 있나요? 그렇게 된 계기나 이유가 있나요.,일이 원하는 대로 진행될 것이라는 자신감이 있어요.,일이 원하는 대로 진행될 것이라는 자신감이 없어요.


In [4]:
model = AutoModel.from_pretrained("klue/roberta-large")
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [5]:
answer_map = {'<그렇다>':0, '<그렇자>':0,
              '<중립>':1, '<중랍>':1, '<중간>':1, '<보통>':1,
              '<어렵다>':2, '<아니다>':2, ',<아니다>':2, '<아니요>':2, '<아니오>':2}
def combine(df):
    list = []
    for i in range(len(df)):
        div = df.Answer[i].index('>')+1
        short_answer = df.Answer[i][:div].replace(' ', '')
        long_answer = df.Answer[i][div:]
        answer_code = answer_map[short_answer]
        q = df.Q_number[i]
        prefix = {0:dq.Positive[q], 1:'', 2:dq.Negative[q]}[answer_code]
        list.append([prefix, long_answer])
    return list

In [8]:
comb = combine(df)
prefix = [t[0] for t in comb]
answer = [t[1] for t in comb]

In [10]:
max_length = model.config.max_position_embeddings
prefix_t = tokenizer(prefix, max_length=max_length, return_tensors='pt', padding=True)
answer_t = tokenizer(answer, max_length=max_length, return_tensors='pt', padding=True)

C:\Program Files\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:2364: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [16]:
answer_t.input_ids.shape

torch.Size([11520, 276])

In [17]:
class PrefixAnswer(Dataset):
    def __init__(self, prefix, answer):
        self.prefix = prefix
        self.answer = answer

    def __len__(self):
        return len(self.prefix['input_ids'])

    def __getitem__(self, idx):
        prefix = {k:v[idx] for k,v in self.prefix.items()}
        answer = {k:v[idx] for k,v in self.answer.items()}
        return prefix, answer

In [18]:
ds = PrefixAnswer(prefix_t, answer_t)
dl = DataLoader(ds, batch_size=32, shuffle=False)

In [19]:
similar = nn.CosineSimilarity()

In [24]:
def forward(model, dl):
    pre_l = []
    ans_l = []
    correct = []
    model.eval()
    for pre, ans in tqdm.tqdm(dl):
        with torch.no_grad():
            pre_out = model(**pre, output_hidden_states=True)
            ans_out = model(**ans, output_hidden_states=True)
        pre_state = pre_out.last_hidden_state[:,0,:]
        ans_state = ans_out.last_hidden_state[:,0,:]
        pre_l.append(pre_state)
        ans_l.append(ans_state)
        sim = similar(pre_state, ans_state)
        correct.append(sim)
        print(sim)
    return torch.cat(pre_l), torch.cat(ans_l), torch.cat(correct)

In [25]:
result = forward(model, dl)

  0%|          | 1/360 [00:30<3:01:49, 30.39s/it]

tensor([0.9906, 0.9670, 0.9780, 0.9411, 0.9704, 0.9743, 0.9735, 0.9746, 0.9711,
        0.9628, 0.9794, 0.9936, 0.9670, 0.9634, 0.9622, 0.9767, 0.9885, 0.9636,
        0.9767, 0.9750, 0.9698, 0.9725, 0.9611, 0.9731, 0.9726, 0.9911, 0.9693,
        0.9720, 0.9813, 0.9551, 0.9674, 0.9926])


  1%|          | 2/360 [01:04<3:14:28, 32.59s/it]

tensor([0.9746, 0.9947, 0.9933, 0.9870, 0.9800, 0.9935, 0.9635, 0.9542, 0.9765,
        0.9552, 0.9745, 0.9714, 0.9775, 0.9924, 0.9736, 0.9610, 0.9698, 0.9877,
        0.9913, 0.9791, 0.9730, 0.9863, 0.9778, 0.9692, 0.9800, 0.9586, 0.9522,
        0.9818, 0.9597, 0.9794, 0.9702, 0.9862])


  1%|          | 2/360 [01:09<3:27:31, 34.78s/it]


KeyboardInterrupt: 

In [50]:
def full_run(t1, t2):
    t = [t1, t2]
    token = [tokenizer([x], max_length=max_length, return_tensors='pt', padding=True) for x in t]
    embed = [model(**x, output_hidden_states=True) for x in token]
    state = [x.last_hidden_state[:,0,:] for x in embed]
    return state

In [51]:
result = full_run('자유 시간 중 상당 부분을 다양한 관심사를 탐구하는 데 할애해요.', '자유 시간 중 상당 부분을 다양한 관심사를 탐구하는 데 할애하지 않아요.')

C:\Program Files\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:2364: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
